In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
ratings = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv')

In [4]:
movies = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv')

In [ ]:
#Sprawdź:
# - Liczbę wszystkich ocen
# - Liczbę filmów w bazie
# - Liczbę użytkowników
# - Średnią liczbę ocen na użytkownika
# - Średnią liczbę ocen na film
# - Rozkład ocen

In [5]:
print('Liczba wszystkich ocen:', ratings.shape[0])
print('Liczba filmów w bazie:', movies.shape[0])
print('Liczba użytkowników:', ratings['userId'].nunique())
print('Średnia liczba ocen na użytkownika:', ratings.shape[0] / ratings['userId'].nunique())
print('Średnia liczba ocen na film:', ratings.shape[0] / movies.shape[0])
print('Rozkład ocen:')
print(ratings['rating'].value_counts())


Liczba wszystkich ocen: 100836
Liczba filmów w bazie: 9742
Liczba użytkowników: 610
Średnia liczba ocen na użytkownika: 165.30491803278687
Średnia liczba ocen na film: 10.350646684459043
Rozkład ocen:
rating
4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: count, dtype: int64
